In [302]:
from __future__ import division
import pandas as pd
import numpy as np

cols_paqs = ['ts', 'temp', 'rh']
cols_airport = ['valid', 'tmpf', 'RH']

f_paqs = '..//datasets/10-Aug//BANG_PAQS.csv'
f_airport = '..//datasets//10-Aug//bangalore.csv'

df_paqs = pd.read_csv(f_paqs, usecols=cols_paqs)
df_paqs.head()

,ts,temp,rh
0,1464340928,31.40,43.96
1,1464340958,33.03,46.10
2,1464340987,33.08,46.43
3,1464341017,33.08,46.44
4,1464341047,33.08,46.44


In [303]:
df_airport = pd.read_csv(f_airport, usecols=cols_airport)
df_airport.head()

,valid,tmpf,RH
0,2016-01-01 00:00,66.20,100.00
1,2016-01-01 00:30,66.20,100.00
2,2016-01-01 01:00,66.20,100.00
3,2016-01-01 01:30,64.40,100.00
4,2016-01-01 02:00,66.20,100.00


In [304]:
df_paqs.dtypes

ts        int64
temp    float64
rh      float64
dtype: object

In [305]:
df_airport.dtypes

valid    object
tmpf     object
RH       object
dtype: object

In [306]:
df_paqs = df_paqs.rename(columns={'temp': 'Temp_paqs', 'rh': 'RH_paqs'})
df_paqs.head()

,ts,Temp_paqs,RH_paqs
0,1464340928,31.40,43.96
1,1464340958,33.03,46.10
2,1464340987,33.08,46.43
3,1464341017,33.08,46.44
4,1464341047,33.08,46.44


In [307]:
df_airport = df_airport.rename(columns={'valid': 'ts', 'tmpf': 'Temp_airport', 'RH': 'RH_airport'})
df_airport.head()

,ts,Temp_airport,RH_airport
0,2016-01-01 00:00,66.20,100.00
1,2016-01-01 00:30,66.20,100.00
2,2016-01-01 01:00,66.20,100.00
3,2016-01-01 01:30,64.40,100.00
4,2016-01-01 02:00,66.20,100.00


In [308]:
df_paqs['ts'] = pd.to_datetime(df_paqs['ts'], unit='s')
df_paqs.head()

,ts,Temp_paqs,RH_paqs
0,2016-05-27 09:22:08,31.40,43.96
1,2016-05-27 09:22:38,33.03,46.10
2,2016-05-27 09:23:07,33.08,46.43
3,2016-05-27 09:23:37,33.08,46.44
4,2016-05-27 09:24:07,33.08,46.44


In [309]:
df_paqs.dtypes

ts           datetime64[ns]
Temp_paqs           float64
RH_paqs             float64
dtype: object

In [310]:
df_paqs = df_paqs.set_index(['ts'])

In [311]:
df_paqs.head()

,Temp_paqs,RH_paqs
ts,,
2016-05-27 09:22:08,31.40,43.96
2016-05-27 09:22:38,33.03,46.10
2016-05-27 09:23:07,33.08,46.43
2016-05-27 09:23:37,33.08,46.44
2016-05-27 09:24:07,33.08,46.44


In [312]:
df_paqs = df_paqs.tz_localize('UTC').tz_convert('Asia/Kolkata')
df_paqs.head()

,Temp_paqs,RH_paqs
ts,,
2016-05-27 14:52:08+05:30,31.40,43.96
2016-05-27 14:52:38+05:30,33.03,46.10
2016-05-27 14:53:07+05:30,33.08,46.43
2016-05-27 14:53:37+05:30,33.08,46.44
2016-05-27 14:54:07+05:30,33.08,46.44


In [313]:
df_airport['ts'] = pd.to_datetime(df_airport['ts'], format='%Y-%m-%d %H:%M')
df_airport.head()

,ts,Temp_airport,RH_airport
0,2016-01-01 00:00:00,66.20,100.00
1,2016-01-01 00:30:00,66.20,100.00
2,2016-01-01 01:00:00,66.20,100.00
3,2016-01-01 01:30:00,64.40,100.00
4,2016-01-01 02:00:00,66.20,100.00


In [314]:
df_airport.dtypes

ts              datetime64[ns]
Temp_airport            object
RH_airport              object
dtype: object

In [315]:
df_airport = df_airport.set_index(['ts'])
df_airport.head()

,Temp_airport,RH_airport
ts,,
2016-01-01 00:00:00,66.20,100.00
2016-01-01 00:30:00,66.20,100.00
2016-01-01 01:00:00,66.20,100.00
2016-01-01 01:30:00,64.40,100.00
2016-01-01 02:00:00,66.20,100.00


In [316]:
df_airport.dtypes

Temp_airport    object
RH_airport      object
dtype: object

In [317]:
df_airport = df_airport[df_airport.Temp_airport != 'M']
df_airport = df_airport[df_airport.RH_airport != 'M']
df_airport['Temp_airport'] = df_airport['Temp_airport'].apply(lambda x: ((float(x)-32)*(5/9)))
df_airport['RH_airport'] = df_airport['RH_airport'].apply(lambda x: float(x))
df_airport.head()

,Temp_airport,RH_airport
ts,,
2016-01-01 00:00:00,19.0,100.0
2016-01-01 00:30:00,19.0,100.0
2016-01-01 01:00:00,19.0,100.0
2016-01-01 01:30:00,18.0,100.0
2016-01-01 02:00:00,19.0,100.0


In [318]:
df_airport.dtypes

Temp_airport    float64
RH_airport      float64
dtype: object

In [319]:
df_airport = df_airport.tz_localize('Asia/Kolkata')
df_paqs = df_paqs.resample('1H').mean()
df_airport = df_airport.resample('1H').mean()
df_paqs = df_paqs.reset_index()
df_airport = df_airport.reset_index()
df_merged = pd.merge(df_paqs, df_airport, on='ts')
df_airport = df_airport.set_index(['ts'])
df_paqs = df_paqs.set_index(['ts'])
df_merged = df_merged.set_index(['ts'])

In [320]:
df_paqs = df_paqs.groupby([df_paqs.index.month, df_paqs.index.hour]).mean()

In [321]:
df_paqs.head()

Temp_paqs    RH_paqs
1 0  22.301684  66.430875
  1  22.698600  64.493302
  2  23.520563  61.566777
  3  23.727815  61.023821
  4  24.314214  58.438189

In [322]:
df_airport = df_airport.groupby([df_airport.index.month, df_airport.index.hour]).mean()

In [323]:
df_airport.head()

Temp_airport  RH_airport
1 0     16.954802   99.485085
  1     16.785714   99.724464
  2     17.621212   99.134424
  3     19.994536   93.237732
  4     22.814516   72.741774

In [324]:
#df_airport = df_airport.reset_index()
#df_paqs = df_paqs.reset_index()

In [325]:
#merge = pd.merge(df_paqs, df_airport, on='index')

In [326]:
df_merged = df_merged.groupby([df_merged.index.month, df_merged.index.hour]).mean()
df_merged.head()

Temp_paqs    RH_paqs  Temp_airport  RH_airport
1 0  22.301684  66.430875     15.216667   99.390000
  1  22.698600  64.493302     15.053571   99.776429
  2  23.520563  61.566777     16.077381   99.021726
  3  23.727815  61.023821     18.198925   93.019731
  4  24.314214  58.438189     20.790323   73.840323

In [327]:
df_merged.describe()

,Temp_paqs,RH_paqs,Temp_airport,RH_airport
count,288.000000,288.000000,288.000000,288.000000
mean,26.817306,57.109773,23.712235,69.569664
std,3.711302,13.739221,4.406110,22.973296
min,20.208184,22.560558,13.428571,19.661964
25%,24.211649,47.082349,20.932143,51.682540
50%,26.332041,58.578392,23.644892,72.021034
75%,28.835924,68.293589,26.844643,90.930049
max,39.184940,79.975599,34.714286,100.000000


In [328]:
df_merged.to_csv('..//datasets//10-Aug//mergedhourly.csv')